# Chapter 6 Homework: 6H3 - 6H7

Working through problems 6H3 to 6H7 from Statistical Rethinking using the **foxes dataset**.

## Key Variables:
- **W**: Body weight
- **A**: Territory area/size
- **F**: Food availability (avgfood)
- **G**: Group size (groupsize)

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent.parent))
from src.quap import quap, QuapResult

plt.style.use('default')
%matplotlib inline

np.random.seed(42)

print('✓ Imports loaded')

## Load and Explore the Foxes Data

The foxes dataset contains information about urban foxes.

In [ ]:
# Load foxes data
url = "https://raw.githubusercontent.com/rmcelreath/rethinking/master/data/foxes.csv"
foxes = pd.read_csv(url, sep=";")

print(f"Dataset shape: {foxes.shape}")
print(f"\nColumns: {foxes.columns.tolist()}")
print(f"\nFirst few rows:")
foxes.head()

In [ ]:
# Check data info
print("Data summary:")
print(foxes.info())
print("\nDescriptive statistics:")
foxes.describe()

In [ ]:
# Extract and standardize key variables
W = foxes['weight'].values          # Body weight
A = foxes['area'].values            # Territory area
F = foxes['avgfood'].values         # Average food availability
G = foxes['groupsize'].values       # Group size

# Standardize all variables
W_std = (W - W.mean()) / W.std()
A_std = (A - A.mean()) / A.std()
F_std = (F - F.mean()) / F.std()
G_std = (G - G.mean()) / G.std()

print("Variables standardized:")
print(f"  W (weight): {W.mean():.3f} ± {W.std():.3f}")
print(f"  A (area): {A.mean():.3f} ± {A.std():.3f}")
print(f"  F (avgfood): {F.mean():.3f} ± {F.std():.3f}")
print(f"  G (groupsize): {G.mean():.3f} ± {G.std():.3f}")

## Understand the DAG

Based on Chapter 6, the typical DAG for foxes is:

```
F (Food) → G (Group size) → W (Weight)
F → A (Area) → W
```

Or more specifically:
- **F → A**: More food leads to larger territories
- **F → G**: More food attracts more foxes (larger groups)
- **G → W**: Larger groups mean less food per fox (negative effect on weight)
- **A → W**: Larger area means more food per fox (positive effect on weight)

Let's visualize these relationships.

In [ ]:
# Visualize relationships
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Plot 1: W vs F
ax = axes[0, 0]
ax.scatter(F, W, s=60, alpha=0.6, edgecolor='black')
ax.set_xlabel('Food (F)', fontsize=11)
ax.set_ylabel('Weight (W)', fontsize=11)
ax.set_title(f'Weight ~ Food\nr = {np.corrcoef(W, F)[0,1]:.3f}', 
             fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

# Plot 2: W vs A
ax = axes[0, 1]
ax.scatter(A, W, s=60, alpha=0.6, edgecolor='black', color='green')
ax.set_xlabel('Area (A)', fontsize=11)
ax.set_ylabel('Weight (W)', fontsize=11)
ax.set_title(f'Weight ~ Area\nr = {np.corrcoef(W, A)[0,1]:.3f}', 
             fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

# Plot 3: W vs G
ax = axes[0, 2]
ax.scatter(G, W, s=60, alpha=0.6, edgecolor='black', color='orange')
ax.set_xlabel('Group Size (G)', fontsize=11)
ax.set_ylabel('Weight (W)', fontsize=11)
ax.set_title(f'Weight ~ Group Size\nr = {np.corrcoef(W, G)[0,1]:.3f}', 
             fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

# Plot 4: A vs F
ax = axes[1, 0]
ax.scatter(F, A, s=60, alpha=0.6, edgecolor='black', color='purple')
ax.set_xlabel('Food (F)', fontsize=11)
ax.set_ylabel('Area (A)', fontsize=11)
ax.set_title(f'Area ~ Food\nr = {np.corrcoef(A, F)[0,1]:.3f}', 
             fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

# Plot 5: G vs F
ax = axes[1, 1]
ax.scatter(F, G, s=60, alpha=0.6, edgecolor='black', color='red')
ax.set_xlabel('Food (F)', fontsize=11)
ax.set_ylabel('Group Size (G)', fontsize=11)
ax.set_title(f'Group Size ~ Food\nr = {np.corrcoef(G, F)[0,1]:.3f}', 
             fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

# Plot 6: G vs A
ax = axes[1, 2]
ax.scatter(A, G, s=60, alpha=0.6, edgecolor='black', color='brown')
ax.set_xlabel('Area (A)', fontsize=11)
ax.set_ylabel('Group Size (G)', fontsize=11)
ax.set_title(f'Group Size ~ Area\nr = {np.corrcoef(G, A)[0,1]:.3f}', 
             fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Problem 6H3: Causal Effect of W → A

**Question**: What is the **causal effect of body weight (W) on territory area (A)**?

**DAG Analysis**:
```
F → A
F → G → W
A → W
```

**Question**: Does W cause A, or does A cause W?

From the biological story:
- Food (F) determines both area and group size
- Area (A) affects weight (W): larger territory = more food per fox
- Group size (G) affects weight (W): more foxes = less food per fox

**So the causal direction is A → W, not W → A!**

If we want to estimate the (likely non-existent or reverse) effect of W → A, we need to think about what confounds this relationship.

In [ ]:
# For W → A, what should we condition on?
# The problem is that this is likely the WRONG causal direction
# But if forced to estimate it, we'd need to close backdoor paths

print("6H3: Causal effect of W → A")
print("="*70)
print("\nDAG structure:")
print("  F → A → W ← G ← F")
print("\nBackdoor paths from W to A:")
print("  1. W ← G ← F → A")
print("  2. W ← A (direct path - this IS the effect we want!)")
print("\nTo estimate W → A (reverse direction):")
print("  - Need to condition on F and G to block backdoor paths")
print("  - But this is problematic because A actually causes W!")
print("\n⚠️ Note: This is likely asking us to recognize the causal direction")
print("   is actually A → W, not W → A!")

Let's fit models to explore this:

In [ ]:
# Model 1: A ~ W (bivariate)
def neg_log_posterior_A_W(params):
    alpha, beta_W, log_sigma = params
    sigma = np.exp(log_sigma)
    mu = alpha + beta_W * W_std
    log_lik = np.sum(stats.norm.logpdf(A_std, loc=mu, scale=sigma))
    log_prior = (stats.norm.logpdf(alpha, 0, 0.2) +
                 stats.norm.logpdf(beta_W, 0, 0.5) +
                 stats.expon.logpdf(sigma, scale=1))
    return -(log_lik + log_prior + log_sigma)

m_A_W = quap(neg_log_posterior_A_W, [0, 0, np.log(1)],
             ['alpha', 'beta_W', 'log_sigma'])
m_A_W.transform_param('log_sigma', 'sigma', np.exp)

print("Model: A ~ W (bivariate)")
print("="*70)
m_A_W.summary()

In [ ]:
# Model 2: A ~ W + F + G (conditioning)
def neg_log_posterior_A_WFG(params):
    alpha, beta_W, beta_F, beta_G, log_sigma = params
    sigma = np.exp(log_sigma)
    mu = alpha + beta_W * W_std + beta_F * F_std + beta_G * G_std
    log_lik = np.sum(stats.norm.logpdf(A_std, loc=mu, scale=sigma))
    log_prior = (stats.norm.logpdf(alpha, 0, 0.2) +
                 stats.norm.logpdf(beta_W, 0, 0.5) +
                 stats.norm.logpdf(beta_F, 0, 0.5) +
                 stats.norm.logpdf(beta_G, 0, 0.5) +
                 stats.expon.logpdf(sigma, scale=1))
    return -(log_lik + log_prior + log_sigma)

m_A_WFG = quap(neg_log_posterior_A_WFG, [0, 0, 0, 0, np.log(1)],
               ['alpha', 'beta_W', 'beta_F', 'beta_G', 'log_sigma'])
m_A_WFG.transform_param('log_sigma', 'sigma', np.exp)

print("Model: A ~ W + F + G (conditioning)")
print("="*70)
m_A_WFG.summary()

---

## Problem 6H4: Causal Effect of A → W

**Question**: What is the **causal effect of territory area (A) on body weight (W)**?

This is the correct causal direction!

**DAG**:
```
F → A → W ← G ← F
```

**Backdoor paths from A to W**:
1. A ← F → G → W

**Solution**: Condition on **F** to block the backdoor path.
- Do NOT condition on G (it's a pipe: A ← F → G → W)
- Conditioning on G would open a path and create bias

In [ ]:
print("6H4: Causal effect of A → W")
print("="*70)
print("\nBackdoor paths from A to W:")
print("  A ← F → G → W")
print("\nSolution: Condition on F only")
print("  ✓ This blocks the backdoor path")
print("  ✗ Do NOT condition on G (descendant of F on the path)")

In [ ]:
# Model: W ~ A + F
def neg_log_posterior_W_AF(params):
    alpha, beta_A, beta_F, log_sigma = params
    sigma = np.exp(log_sigma)
    mu = alpha + beta_A * A_std + beta_F * F_std
    log_lik = np.sum(stats.norm.logpdf(W_std, loc=mu, scale=sigma))
    log_prior = (stats.norm.logpdf(alpha, 0, 0.2) +
                 stats.norm.logpdf(beta_A, 0, 0.5) +
                 stats.norm.logpdf(beta_F, 0, 0.5) +
                 stats.expon.logpdf(sigma, scale=1))
    return -(log_lik + log_prior + log_sigma)

m_W_AF = quap(neg_log_posterior_W_AF, [0, 0, 0, np.log(1)],
              ['alpha', 'beta_A', 'beta_F', 'log_sigma'])
m_W_AF.transform_param('log_sigma', 'sigma', np.exp)

print("Model: W ~ A + F (correct conditioning)")
print("="*70)
m_W_AF.summary()
print("\n✓ beta_A is the causal effect of area on weight")

---

## Problem 6H5: Causal Effect of G → W

**Question**: What is the **causal effect of group size (G) on body weight (W)**?

**DAG**:
```
F → G → W ← A ← F
```

**Backdoor paths from G to W**:
1. G ← F → A → W

**Solution**: Condition on **F** to block the backdoor path.
- Do NOT condition on A (it's a mediator on one path and would block part of the effect)

In [ ]:
print("6H5: Causal effect of G → W")
print("="*70)
print("\nBackdoor paths from G to W:")
print("  G ← F → A → W")
print("\nSolution: Condition on F only")
print("  ✓ This blocks the backdoor path")
print("  ✗ Do NOT condition on A (it's on a different causal path)")

In [ ]:
# Model: W ~ G + F
def neg_log_posterior_W_GF(params):
    alpha, beta_G, beta_F, log_sigma = params
    sigma = np.exp(log_sigma)
    mu = alpha + beta_G * G_std + beta_F * F_std
    log_lik = np.sum(stats.norm.logpdf(W_std, loc=mu, scale=sigma))
    log_prior = (stats.norm.logpdf(alpha, 0, 0.2) +
                 stats.norm.logpdf(beta_G, 0, 0.5) +
                 stats.norm.logpdf(beta_F, 0, 0.5) +
                 stats.expon.logpdf(sigma, scale=1))
    return -(log_lik + log_prior + log_sigma)

m_W_GF = quap(neg_log_posterior_W_GF, [0, 0, 0, np.log(1)],
              ['alpha', 'beta_G', 'beta_F', 'log_sigma'])
m_W_GF.transform_param('log_sigma', 'sigma', np.exp)

print("Model: W ~ G + F (correct conditioning)")
print("="*70)
m_W_GF.summary()
print("\n✓ beta_G is the causal effect of group size on weight")

---

## Problem 6H6: Your Own Research Question

**Question**: Consider your own research question. Draw a DAG representing the causal relationships.

*This is an open-ended question for you to think about your own research interests.*

In [ ]:
print("6H6: Design your own DAG for a research question")
print("="*70)
print("\nExample: Effect of study time on exam performance")
print("\nVariables:")
print("  - S: Study time")
print("  - E: Exam score")
print("  - I: Intelligence/ability")
print("  - M: Motivation")
print("\nDAG:")
print("  I → S → E ← M → S")
print("  I → E")
print("\nTo estimate S → E, condition on I and M")

---

## Problem 6H7: Complex DAG Analysis

**Question**: Given a DAG with grandparents → parents → offspring structure, answer questions about:
1. Which pairs of variables are conditionally independent?
2. What to condition on for various causal effects?

*Need to see the specific DAG structure from the problem to answer this properly.*

In [ ]:
print("6H7: Complex DAG with grandparents/parents/offspring")
print("="*70)
print("\nNeed to analyze the specific DAG structure shown in the problem.")
print("\nGeneral approach:")
print("  1. Identify all paths between variables")
print("  2. Determine which are backdoor paths")
print("  3. Find minimal conditioning sets to block backdoor paths")
print("  4. Check for colliders (don't condition on them!)")

---

## Summary

**Key insights from foxes problems**:

1. **6H3 (W → A)**: This asks for the effect in the WRONG causal direction. The real effect is A → W. To estimate the reverse, we'd need to condition on F and G, but this is problematic.

2. **6H4 (A → W)**: Correct causal direction. Condition on **F only** to block the backdoor path A ← F → G → W.

3. **6H5 (G → W)**: Condition on **F only** to block the backdoor path G ← F → A → W.

**General lesson**: 
- Always start with the DAG
- Identify backdoor paths
- Condition on variables to block backdoors
- Don't condition on mediators or colliders
- Make sure you have the causal direction right!